In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

actions = ['come', 'away', 'spin']
seq_length = 30

model = load_model('/Users/jeongtaewon/Desktop/AI/models/model2.h5')

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

# w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# out = cv2.VideoWriter('input.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))
# out2 = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))

seq = []
action_seq = []

while cap.isOpened():
    ret, img = cap.read()
    img0 = img.copy()

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            print(res.landmark)
            print("--------")
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
            v = v2 - v1 # [20, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree

            d = np.concatenate([joint.flatten(), angle])

            seq.append(d)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) < seq_length:
                continue

            input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)

            y_pred = model.predict(input_data).squeeze()
            # 결과를 뽑아내서 y_pred에 저장
            i_pred = int(np.argmax(y_pred))
            # 어떤인덱스인지 , conf를 뽑아낸다.
            conf = y_pred[i_pred]

            if conf < 0.9:
                continue
            # 0.9보다 낮다면 제스처를 취하지 않았다고 인지하고 버린다.
            action = actions[i_pred]
            action_seq.append(action)
            #seq에 저장한다.

            if len(action_seq) < 3:
                continue

            this_action = '?'
            if action_seq[-1] == action_seq[-2] == action_seq[-3]:
                this_action = action

            #마지막 3개의 액션이 모두 같다면 유효한 액션으로 판단후 화면에 띄운다. 그렇지 않다면 "?"를 출력.

            cv2.putText(img, f'{this_action.upper()}', org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

    # out.write(img0)
    # out2.write(img)
    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

[x: 0.39005857706069946
y: 0.7836427092552185
z: 8.28526154350584e-08
, x: 0.40913790464401245
y: 0.7331281900405884
z: 0.0014968159375712276
, x: 0.442810982465744
y: 0.7091643810272217
z: -0.0058751776814460754
, x: 0.4734574556350708
y: 0.7083597183227539
z: -0.013058659620583057
, x: 0.4969068169593811
y: 0.7177594900131226
z: -0.02033015340566635
, x: 0.44525808095932007
y: 0.6802811622619629
z: -0.026068583130836487
, x: 0.4828141927719116
y: 0.7048251032829285
z: -0.034383416175842285
, x: 0.49814102053642273
y: 0.7342323064804077
z: -0.03555207699537277
, x: 0.5053016543388367
y: 0.7586869597434998
z: -0.03610553219914436
, x: 0.440746933221817
y: 0.7113783955574036
z: -0.033323727548122406
, x: 0.4801056981086731
y: 0.7396044135093689
z: -0.042024966329336166
, x: 0.5012313723564148
y: 0.7670224905014038
z: -0.0436224639415741
, x: 0.5134121775627136
y: 0.7907556891441345
z: -0.04554227739572525
, x: 0.4351641833782196
y: 0.7486774921417236
z: -0.03874317184090614
, x: 0.47262

[x: 0.38344115018844604
y: 0.7786626219749451
z: 8.476315116467958e-08
, x: 0.40959128737449646
y: 0.7373145818710327
z: 0.00986551120877266
, x: 0.42454084753990173
y: 0.7000427842140198
z: 0.00818077102303505
, x: 0.43706315755844116
y: 0.6734524369239807
z: 0.0026980703696608543
, x: 0.4474567174911499
y: 0.6556040644645691
z: -0.003708532312884927
, x: 0.4101467430591583
y: 0.6406650543212891
z: -0.0022403548937290907
, x: 0.43184617161750793
y: 0.6233931183815002
z: -0.011406359262764454
, x: 0.4471795856952667
y: 0.6166273951530457
z: -0.01718672551214695
, x: 0.4608461260795593
y: 0.6123456358909607
z: -0.021036243066191673
, x: 0.39773082733154297
y: 0.6478768587112427
z: -0.013651290908455849
, x: 0.4316190183162689
y: 0.6569728851318359
z: -0.026699313893914223
, x: 0.4520677626132965
y: 0.6850970387458801
z: -0.03050500527024269
, x: 0.46469590067863464
y: 0.7132793664932251
z: -0.031219808384776115
, x: 0.3881433308124542
y: 0.6693745851516724
z: -0.02422833815217018
, x: 0

[x: 0.248994380235672
y: 0.8667989373207092
z: 4.804756486009865e-07
, x: 0.2963470220565796
y: 0.7972668409347534
z: -0.013397944159805775
, x: 0.3299017548561096
y: 0.710199236869812
z: -0.015232164412736893
, x: 0.3536439538002014
y: 0.6361300945281982
z: -0.018261155113577843
, x: 0.37917718291282654
y: 0.5937541127204895
z: -0.02128518931567669
, x: 0.292397677898407
y: 0.598721444606781
z: 0.009092682972550392
, x: 0.3023363947868347
y: 0.5114901661872864
z: 0.0016972244484350085
, x: 0.30519264936447144
y: 0.45505255460739136
z: -0.009068042039871216
, x: 0.3074512183666229
y: 0.4055113196372986
z: -0.01752306893467903
, x: 0.25781717896461487
y: 0.5919175744056702
z: 0.006585244555026293
, x: 0.25481462478637695
y: 0.49240732192993164
z: 0.002884880406782031
, x: 0.2519686818122864
y: 0.4262980818748474
z: -0.0072623626329004765
, x: 0.2501544654369354
y: 0.36729103326797485
z: -0.01538075041025877
, x: 0.22579695284366608
y: 0.6103537678718567
z: 0.00045816256897524
, x: 0.215

[x: 0.28859826922416687
y: 0.7967455387115479
z: 4.3663635551638436e-07
, x: 0.33226585388183594
y: 0.7336981296539307
z: -0.01891046017408371
, x: 0.3618231415748596
y: 0.6416951417922974
z: -0.026497071608901024
, x: 0.3823583722114563
y: 0.5662908554077148
z: -0.03360169380903244
, x: 0.40736955404281616
y: 0.5215715169906616
z: -0.040816646069288254
, x: 0.31221845746040344
y: 0.5398799777030945
z: -0.010545737110078335
, x: 0.31774717569351196
y: 0.44966140389442444
z: -0.024326955899596214
, x: 0.3211935758590698
y: 0.3913896381855011
z: -0.037233274430036545
, x: 0.3248119354248047
y: 0.3416787385940552
z: -0.04688536375761032
, x: 0.27708709239959717
y: 0.5396051406860352
z: -0.012916825711727142
, x: 0.270042359828949
y: 0.43324846029281616
z: -0.024206167086958885
, x: 0.2660367488861084
y: 0.3630496859550476
z: -0.03598723188042641
, x: 0.2632400691509247
y: 0.30178511142730713
z: -0.04451858997344971
, x: 0.24724535644054413
y: 0.5635076761245728
z: -0.018316324800252914
, 

[x: 0.300327330827713
y: 0.891158938407898
z: 2.2998000304141897e-07
, x: 0.33436596393585205
y: 0.8141860961914062
z: -0.02959607169032097
, x: 0.34053492546081543
y: 0.7077711224555969
z: -0.03891922906041145
, x: 0.3358326256275177
y: 0.6249768137931824
z: -0.047029707580804825
, x: 0.3330727815628052
y: 0.5558880567550659
z: -0.0544736385345459
, x: 0.2709238529205322
y: 0.605710506439209
z: -0.013446850702166557
, x: 0.2530224025249481
y: 0.528032660484314
z: -0.03227164223790169
, x: 0.25150078535079956
y: 0.48464518785476685
z: -0.049203988164663315
, x: 0.2592335343360901
y: 0.46432679891586304
z: -0.060096513479948044
, x: 0.23538392782211304
y: 0.6280014514923096
z: -0.010767337866127491
, x: 0.2029481828212738
y: 0.5522022843360901
z: -0.029073096811771393
, x: 0.2027006298303604
y: 0.509387195110321
z: -0.04827524721622467
, x: 0.2196894884109497
y: 0.5013430118560791
z: -0.061414606869220734
, x: 0.2113885134458542
y: 0.6741536855697632
z: -0.012905273586511612
, x: 0.1753

[x: 0.2941027879714966
y: 0.8384701013565063
z: 4.508972608618933e-07
, x: 0.3440600335597992
y: 0.7959498167037964
z: -0.020559197291731834
, x: 0.3836714029312134
y: 0.7133430242538452
z: -0.026490556076169014
, x: 0.41901373863220215
y: 0.6532609462738037
z: -0.03165481612086296
, x: 0.4507019519805908
y: 0.6369349360466003
z: -0.03647531569004059
, x: 0.3485041856765747
y: 0.5893285870552063
z: -0.007209480740129948
, x: 0.3673069477081299
y: 0.5018647909164429
z: -0.014848994091153145
, x: 0.37587881088256836
y: 0.44633710384368896
z: -0.023774346336722374
, x: 0.3814895749092102
y: 0.3994417190551758
z: -0.031186947599053383
, x: 0.31385886669158936
y: 0.5771063566207886
z: -0.006531010381877422
, x: 0.3182481527328491
y: 0.4740813672542572
z: -0.011131876148283482
, x: 0.31986379623413086
y: 0.4109798073768616
z: -0.018407005816698074
, x: 0.3197457492351532
y: 0.3592745065689087
z: -0.024139080196619034
, x: 0.2819232940673828
y: 0.5898340940475464
z: -0.009130188263952732
, x:

[x: 0.2968592941761017
y: 0.852527916431427
z: 4.181289057214599e-07
, x: 0.3410758376121521
y: 0.8053159713745117
z: -0.01726279780268669
, x: 0.37570977210998535
y: 0.722204327583313
z: -0.024480368942022324
, x: 0.40103933215141296
y: 0.6527189016342163
z: -0.03140456601977348
, x: 0.4282786548137665
y: 0.6106570959091187
z: -0.03808138146996498
, x: 0.3416211009025574
y: 0.595147967338562
z: -0.00984878558665514
, x: 0.3558383285999298
y: 0.5074883103370667
z: -0.024209721013903618
, x: 0.3658483028411865
y: 0.452225923538208
z: -0.03721029311418533
, x: 0.37517935037612915
y: 0.40592870116233826
z: -0.0468812994658947
, x: 0.30830374360084534
y: 0.5841683149337769
z: -0.013585096225142479
, x: 0.3124910891056061
y: 0.47624653577804565
z: -0.025811629369854927
, x: 0.31847721338272095
y: 0.40490126609802246
z: -0.03803693875670433
, x: 0.3241446316242218
y: 0.3452072739601135
z: -0.04657718911767006
, x: 0.27786633372306824
y: 0.5998218059539795
z: -0.02024945244193077
, x: 0.26910

[x: 0.30565428733825684
y: 0.868739128112793
z: 3.31339236936401e-07
, x: 0.3496975302696228
y: 0.8044091463088989
z: -0.017444157972931862
, x: 0.3761100471019745
y: 0.7090879082679749
z: -0.021219152957201004
, x: 0.3893044590950012
y: 0.6264230608940125
z: -0.026172174140810966
, x: 0.3987012207508087
y: 0.5599322319030762
z: -0.030028492212295532
, x: 0.3327285349369049
y: 0.5995115041732788
z: 0.011708531528711319
, x: 0.33407843112945557
y: 0.5139994621276855
z: -0.0005288854008540511
, x: 0.3423909544944763
y: 0.4769308567047119
z: -0.015584820881485939
, x: 0.35421186685562134
y: 0.463590532541275
z: -0.025718340650200844
, x: 0.30012786388397217
y: 0.596348226070404
z: 0.00999284815043211
, x: 0.2880231738090515
y: 0.5044805407524109
z: -0.00019545387476682663
, x: 0.29658740758895874
y: 0.4529993534088135
z: -0.016487903892993927
, x: 0.313151478767395
y: 0.4338539242744446
z: -0.02875884808599949
, x: 0.27120909094810486
y: 0.6159271597862244
z: 0.00325999828055501
, x: 0.25

[x: 0.4415491223335266
y: 0.9738556146621704
z: 2.9433834924930125e-07
, x: 0.45689070224761963
y: 0.8963499069213867
z: -0.011546952649950981
, x: 0.443875253200531
y: 0.8333367705345154
z: -0.015076648443937302
, x: 0.4196853041648865
y: 0.7855954766273499
z: -0.013751707039773464
, x: 0.3991454541683197
y: 0.7431330680847168
z: -0.008011763915419579
, x: 0.3852822184562683
y: 0.8141432404518127
z: -0.029528455808758736
, x: 0.3579612672328949
y: 0.7272483706474304
z: -0.036147911101579666
, x: 0.3601115047931671
y: 0.6985046863555908
z: -0.025658149272203445
, x: 0.3734787404537201
y: 0.7069120407104492
z: -0.010860210284590721
, x: 0.3637121617794037
y: 0.8471728563308716
z: -0.019836675375699997
, x: 0.3144124150276184
y: 0.7781458497047424
z: -0.02915194071829319
, x: 0.3068338930606842
y: 0.7281617522239685
z: -0.01602059043943882
, x: 0.31083858013153076
y: 0.7031676769256592
z: 0.001359448186121881
, x: 0.3557237684726715
y: 0.8803757429122925
z: -0.009801993146538734
, x: 0.3

[x: 0.5950919985771179
y: 0.972776472568512
z: 2.1160967378364148e-07
, x: 0.6075770854949951
y: 0.890807569026947
z: -0.008927321061491966
, x: 0.5881356000900269
y: 0.8194857239723206
z: -0.020290063694119453
, x: 0.5620406866073608
y: 0.7702803611755371
z: -0.02836032584309578
, x: 0.5538470149040222
y: 0.7290304899215698
z: -0.03767058625817299
, x: 0.5746638774871826
y: 0.8358922004699707
z: -0.04680937901139259
, x: 0.531961977481842
y: 0.7442458868026733
z: -0.062161363661289215
, x: 0.5123792290687561
y: 0.6988492012023926
z: -0.06696303188800812
, x: 0.4963832497596741
y: 0.6682135462760925
z: -0.06862461566925049
, x: 0.5533621907234192
y: 0.8764730095863342
z: -0.048782072961330414
, x: 0.5128170847892761
y: 0.7774598598480225
z: -0.06501355022192001
, x: 0.488787442445755
y: 0.726004421710968
z: -0.06963906437158585
, x: 0.4692189693450928
y: 0.6898372769355774
z: -0.07195094972848892
, x: 0.5360764861106873
y: 0.9133239984512329
z: -0.04823160916566849
, x: 0.5063203573226